# RAG

In [ ]:
#mistralai/Mixtral-8x7B-Instruct-v0.1

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/mnt/d/Code/tokenGPT/training_data/tokenGPT-KnowledgeBase/blockchain economic systems modelling.pdf")
pages = loader.load()

# Data Extraction Tools

## Uniswap subgraphs data

In [46]:
import pandas as pd
import json
import requests
import requests
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
import numpy as np

from scipy.stats import zscore
from dune_client.client import DuneClient
import datetime
from pytz import utc

In [20]:
def get_uniswap_top_pools(n_pools):
    
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

    # GraphQL query to fetch the pool id and daily volume for the top n_pools
    query = """
    {{
      pools(first: {0}, orderBy: volumeUSD, orderDirection: desc) {{
        id
        volumeUSD
      }}
    }}
    """.format(n_pools)

    # Make the request
    response = requests.post(url, json={'query': query})

    # Get the JSON data from the response
    data = json.loads(response.text)

    # Get the pool IDs from the data
    pool_ids = [pool['id'] for pool in data['data']['pools']]

    return pool_ids

def get_uniswap_pool_day_data(pool_id, number_of_days):
  
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
    query = f"""
    {{
        poolDayDatas(where: {{ pool: "{pool_id}" }}, first: {number_of_days}, orderBy: date, orderDirection: desc) {{
            date
            tick
            sqrtPrice
            liquidity
            volumeUSD
            volumeToken0
            volumeToken1
            tvlUSD
            feesUSD
            close
            open
            low
            high
        }}
    }}
    """
    response = requests.post(url, json={'query': query})
    data = response.json()
    df = pd.DataFrame(data['data']['poolDayDatas'])

    return df

def save_data_to_csv(data, file_path):
    keys = data[0].keys()
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)


def preprocess_uniswap_subgraph_data(data_df):
    data_df['volumeUSD'] = data_df['volumeUSD'].astype(float)
    data_df['volumeToken0'] = data_df['volumeToken0'].astype(float)
    data_df['volumeToken1'] = data_df['volumeToken1'].astype(float)
    data_df['sqrtPrice'] = (data_df['sqrtPrice'].astype(float))
    data_df['liquidity'] = data_df['liquidity'].astype(float)
    data_df['tvlUSD'] = data_df['tvlUSD'].astype(float)
    data_df['feesUSD'] = data_df['feesUSD'].astype(float)
    data_df['close'] = data_df['close'].astype(float)
    data_df['open'] = data_df['open'].astype(float)
    data_df['low'] = data_df['low'].astype(float)
    data_df['high'] = data_df['high'].astype(float)
    
      # Convert date from UNIX timestamp to datetime format
    data_df['date'] = pd.to_datetime(data_df['date'], unit='s')
    # 7 days Rolling averages
    data_df['feesUSD'] = data_df['feesUSD'].rolling(window=7, min_periods=1).mean()

    data_df['date'] = pd.to_datetime(data_df['date'], unit='s')
  
# Check for missing data in the control group
    missing_values = data_df.isnull().sum()
    missing_control = missing_values[missing_values > 0]

    return data_df

def explore_uniswap_pool_data(df):
    #Raw data analysis

    df.plot(x='date', y='sqrtPrice')

    df.plot(x='date', y='feesUSD')
    df.plot(x='date', y='liquidity')

    ax = df.plot(x='date', y='liquidity', color='blue', label='liquidity')
    df.plot(x='date', y='sqrtPrice', color='red', secondary_y=True, ax=ax, label='sqrtPrice')

    ax.set_ylabel('liquidity')
    ax.right_ax.set_ylabel('sqrtPrice')
    plt.title('liquidity and sqrtPrice Over Time')
    plt.show()

    ax = df.plot(x='date', y='liquidity', color='blue', label='liquidity')
    df.plot(x='date', y='feesUSD', color='red', secondary_y=True, ax=ax, label='Fees')

    ax.set_ylabel('liquidity')
    ax.right_ax.set_ylabel('Fees')
    plt.title('liquidity and Fees Over Time')
    plt.show()

    ax = df.plot(x='date', y='volumeToken0', color='blue', label='volumeToken0')
    df.plot(x='date', y='volumeToken1', color='red', secondary_y=True, ax=ax, label='volumeToken1')

    ax.set_ylabel('volumeToken0')
    ax.right_ax.set_ylabel('volumeToken1')
    plt.title('liquidity and Fees Over Time')
    plt.show()

    ax = df.plot(x='date', y='sqrtPrice', color='blue', label='sqrtPrice')
    df.plot(x='date', y='sqrtPrice', color='red', secondary_y=True, ax=ax, label='sqrtPrice')
    ax.set_ylabel('sqrtPrice')
    ax.right_ax.set_ylabel('sqrtPrice')
    plt.title('liquidity and Fees Over Time')
    plt.show()

    #EDA
    plt.hist(df['volumeUSD'])
    plt.show()
    sns.boxplot(x=df['volumeUSD'])
    plt.show()
    plt.scatter(x=df['date'],y=df['volumeUSD'])
    plt.show()
    sns.countplot(df['volumeUSD'])
    plt.show()
    sns.heatmap(df.corr())
    plt.show()
    corr_matrix=df.corr()
    sns.heatmap(corr_matrix,annot=True,cmap='coolwarm')

    #zscore
    df['zscore']=zscore(df['volumeUSD'])
    outliers=df[(df['zscore']>3) | (df['zscore']<-3)]
    print(outliers)

    #transformations
    df['log_transformed']=np.log(df['volumeUSD'])

    #standardization
    scaler = StandardScaler()
    df['standardized_volume'] = scaler.fit_transform(df['volumeUSD'].values.reshape(-1, 1))

    return True

In [47]:
    
def get_uniswap_top_pools_daily_data(n_pools=5,days=30):
    def get_uniswap_pool_daily_data(pool_id, number_of_days):
        url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
        query = f"""
        {{
            poolDayDatas(where: {{ pool: "{pool_id}" }}, first: {number_of_days}, orderBy: date, orderDirection: desc) {{
                date
                sqrtPrice
                liquidity
                volumeUSD
                tvlUSD
                feesUSD
                pool {{
                    token0 {{
                        symbol
                    }}
                    token1 {{
                        symbol
                    }}
                    feeTier
                }}
            }}
        }}
        """
        response = requests.post(url, json={'query': query})
        data = response.json()
        
        for poolDayData in data['data']['poolDayDatas']:
            poolDayData['pool_name'] = poolDayData['pool']['token0']['symbol'] + "/" + poolDayData['pool']['token1']['symbol'] + "(" + str(poolDayData['pool']['feeTier']) + ")"
            poolDayData['feesUSD'] = float(poolDayData['feesUSD'])
            poolDayData['tvlUSD'] = float(poolDayData['tvlUSD'])
            poolDayData['volumeUSD'] = float(poolDayData['volumeUSD'])
            poolDayData['date'] = pd.to_datetime(poolDayData['date'], unit='s')

        return data['data']['poolDayDatas']


    pool_ids=get_uniswap_top_pools(n_pools)
    uniswap_pools_daily_data={}
    for pool_id in pool_ids:
        uniswap_data = get_uniswap_pool_daily_data(pool_id, days)
        df = pd.DataFrame(uniswap_data)
        df.drop('pool', axis=1, inplace=True)
        uniswap_pools_daily_data[pool_id] = df
    return uniswap_pools_daily_data

def calculate_pool_performance_matrices(uniswap_pools_daily_data):
    results = {}
    for pool_id, df in uniswap_pools_daily_data.items():
        
        df = df[(df['feesUSD'] != 0) & (df['tvlUSD'] != 0) & df['feesUSD'].notna() & df['tvlUSD'].notna()]
        
        pool_name = df['pool_name'].iloc[0]
        avg_feeUSD = df['feesUSD'].ewm(span=30).mean().iloc[-1]
        avg_tvl = df['tvlUSD'].ewm(span=30).mean().iloc[-1]
        
        adv = df['volumeUSD'].ewm(span=30).mean().iloc[-1]

        daily_roi = (df['feesUSD'] /df['tvlUSD']).ewm(span=30).mean().iloc[-1]
        monthly_roi = ((1 + daily_roi)**30) - 1
        annual_roi = ((1 + daily_roi)**365) - 1
       
        results[pool_id] = {'pool_name':pool_name,'avg_feesUSD': avg_feeUSD, 'avg_tvlUSD': avg_tvl, 'adv': adv, 'daily_roi': daily_roi,'monthly_roi':monthly_roi,'annual_roi':annual_roi}
    df_rois_uniswap = pd.DataFrame.from_dict(results, orient='index')
    return df_rois_uniswap

In [ ]:
get_uniswap_top_pools(5)
df=get_uniswap_pool_day_data('0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',10)
df=preprocess_uniswap_subgraph_data(df)
explore_uniswap_pool_data(df)
data=get_uniswap_top_pools_daily_data(3,2)
calculate_pool_performance_matrices(data)

## Dune Analytics Data

In [52]:
def fetch_query_result(query_id):
    dune_api_key = "gqBRKclPQMlU9Jm009ikKIrYV4gWhtuq"
    dune = DuneClient(dune_api_key)
    #curl -H "X-Dune-API-Key:" gqBRKclPQMlU9Jm009ikKIrYV4gWhtuq"https://api.dune.com/api/v1/query/3467177/results?limit=1000"
    # Fetch the latest query result
    query_result = dune.get_latest_result(query_id)
    return query_result.result.rows if query_result.result.rows else []


In [53]:
queries = {
    "latest_price": 3466806,
    "all_contract_tx": 3467027,
    "trade_data": 3467093,
    "mydpt_stats":3478340,
    "keys_buy_sell":3468401,
}
fetch_query_result(3466806)

[{'date': '2024-05-16 23:09:00.000 UTC', 'price': 0.014633}]

## ETH Maianet data from Alchemy/Infura

## ETH Maianet data from Etherscan

## Defillama Data

# Analysis Tools

## Lping Vs Lending Analysis

In [21]:
def lping_vs_lending_analysis(usdc_pool_0 = 30_000_000,eth_pool_0 = 16_000,syncswap_apr = 0.11,reactorfusion_usdc_apr = 0.065,reactorfusion_eth_apr = 0.045):
    import numpy as np
    import pandas as pd
    import logging
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_theme()

    logging.basicConfig(level=logging.INFO, format='%(levelname)-8s %(message)s')
    logger = logging.getLogger('Sim')

     # 30 million USDC in SyncSwap pool initially
     # 16 thousand ETH in SyncSwap pool initially
    initial_price = usdc_pool_0 / eth_pool_0
    logger.info(f'Initial SyncSwap ETH/USDC price = {usdc_pool_0 / eth_pool_0}')
    logger.info(f'Pool TVL = ${usdc_pool_0 * 2 / 1e6} million\n')

    eth_lp_amount = 1 # User LPs 10 ETH
    usdc_lp_amount = (usdc_pool_0 / eth_pool_0) * eth_lp_amount
    logger.info(f'User 1 LPs {eth_lp_amount} ETH. User 2 LPs {usdc_lp_amount} USDC.')
    user_pool_share = eth_lp_amount / (eth_pool_0 + eth_lp_amount)
    logger.info(f'Users together own {user_pool_share * 100}% of the total pool')

     # 11% is a conservative APR estimate
     # 4.5% net APR is a high estimate

    # There are multiple ways of generating a random final state. This is a simple one.
    # Allow the final USDC (and ETH) pool amount to be up to 50% different from the current one
    # We sample uniformly from this distribution to deliberately oversample unlikely big changes
    # (since those are the potentially problematic edge cases)
    n = 1000 # Number of samples
    diff_threshold = 0.4

    usdc_pool_f_arr = np.rint(usdc_pool_0 * (2 * np.random.random(n) * diff_threshold - diff_threshold + 1))
    eth_pool_f_arr = np.rint(eth_pool_0 * (2 * np.random.random(n) * diff_threshold - diff_threshold + 1))

    _len = 4
    logger.info('First few simulated end results:')
    logger.info(f'USDC pool amounts = {usdc_pool_f_arr[:_len]}')
    logger.info(f'ETH pool amounts = {eth_pool_f_arr[:_len]}')
    logger.info(f'ETH/USDC prices = {np.rint(usdc_pool_f_arr / eth_pool_f_arr)[:_len]}')
    logger.info(f'Pool TVLs = ${usdc_pool_f_arr[:_len] * 2 / 1e6} million')


    """
    Note that we can break down any change in pool composition into a single (1) add/remove liquidity + (2) swap
    Swaps do not change a user's pool share (since fundamentally there is no change in LP). And they keep x * y = k.
    So we pretend that the add/remove liquidity comes first (makes calculating pool share difference easy)
    and then account for the swap that brings us to our final pool amounts.

    Did some math, here is the result:
    After the add/remove liquidity, x_mid = sqrt(price_0 * k_f) and y_mid = k_f / x_mid
    So (x_mid - x0) liquidity is added of token1 and (y_mid - y0) of token 2 at our initial price
    """
    def simulate(
        pool_token1_initial_amount, pool_token2_initial_amount,
        user_token1_lp_amount, user_token2_lp_amount,
        pool_token1_final_amount, pool_token2_final_amount,
        dex_apr, lending_token1_apr, lending_token2_apr,
        token1_name = "token1", token2_name = "token2"
    ):
        initial_price = pool_token1_initial_amount / pool_token2_initial_amount
        logger.info(f'Initial pool composition: {pool_token1_initial_amount} {token1_name}, '
            f'{pool_token2_initial_amount} {token2_name}')
        logger.info(f'Initial price of {token2_name}/{token1_name} = {initial_price}\n')
        assert(np.isclose(initial_price, user_token1_lp_amount / user_token2_lp_amount))
        
        pool_token1_initial_amount += user_token1_lp_amount
        pool_token2_initial_amount += user_token2_lp_amount
        
        initial_user_pool_share = user_token1_lp_amount / pool_token1_initial_amount
        assert(np.isclose(initial_user_pool_share, user_token2_lp_amount / pool_token2_initial_amount))
        logger.info(f'1. Alice and Bob LP {user_token1_lp_amount} {token1_name} and {user_token2_lp_amount} {token2_name}')
        logger.info(f'   Initially, they own {initial_user_pool_share * 100}% of the total pool')
            
        k_f = pool_token1_final_amount * pool_token2_final_amount
        
        # Pool composition after the add/remove liquidity and before the swap
        pool_token1_intermediate_amount = np.sqrt(initial_price * k_f)
        pool_token2_intermediate_amount = k_f / pool_token1_intermediate_amount
        
        token1_liquidity_delta = pool_token1_intermediate_amount - pool_token1_initial_amount
        token2_liquidity_delta = pool_token2_intermediate_amount - pool_token2_initial_amount
        logger.info(f'2. Other users {"added" if token1_liquidity_delta > 0 else "removed"} liquidity: '
            f'{np.round(np.abs(token1_liquidity_delta), 2)} {token1_name}, '
            f'{np.round(np.abs(token2_liquidity_delta), 2)} {token2_name}')
        
        token1_swap_delta = pool_token1_final_amount - pool_token1_intermediate_amount
        token2_swap_delta = pool_token2_final_amount - pool_token2_intermediate_amount
        final_price = pool_token1_final_amount / pool_token2_final_amount
        logger.info(f'3. Other users {"sold" if token1_swap_delta > 0 else "bought"} '
            f'{np.round(np.abs(token1_swap_delta), 2)} {token1_name} for '
            f'{np.round(np.abs(token2_swap_delta))} {token2_name}\n')
        logger.info(f'Final pool composition: {pool_token1_final_amount} {token1_name}, '
        f'{pool_token2_final_amount} {token2_name}')
        logger.info(f'Final price of {token2_name}/{token1_name} = {np.round(final_price, 2)}')
        final_user_pool_share = user_token1_lp_amount / pool_token1_intermediate_amount
        assert(np.isclose(final_user_pool_share, user_token2_lp_amount / pool_token2_intermediate_amount))
        logger.info(f'Ultimately, the specified LPs own {final_user_pool_share * 100}% of the total pool\n')

        # Specific profit analysis below
        lending_token1_out = user_token1_lp_amount * (1 + lending_token1_apr)
        lending_token2_out = user_token2_lp_amount * (1 + lending_token2_apr)
        logger.info(f'Benchmark = ReactorFusion pools: Assume {lending_token1_apr * 100}% APR on'
            f' {token1_name} and {lending_token2_apr * 100}% APR on {token2_name}')
        logger.info(f'Simply lending in ReactorFusion would yield '
            f'{np.round(lending_token1_out, 2)} {token1_name} and '
            f'{np.round(lending_token2_out, 2)} {token2_name}\n')
        
        logger.info(f'Assume a conservative {dex_apr * 100}% APR on the SyncSwap {token1_name}/{token2_name} pool')
        withdraw_token1_amount = final_user_pool_share * pool_token1_final_amount * (1 + syncswap_apr)
        withdraw_token2_amount = final_user_pool_share * pool_token2_final_amount * (1 + syncswap_apr)
        unbalanced_token1_amount = withdraw_token1_amount
        unbalanced_token2_amount = withdraw_token2_amount
        logger.info(f'Without a rebalancing mechanism, Alice and Bob withdraw '
            f'{np.round(withdraw_token1_amount, 2)} {token1_name}'
            f' and {np.round(withdraw_token2_amount, 2)} {token2_name} from SyncSwap\n')
        
        # Now we will swap token1 for token2 or vice versa so that both sides are the same percentage better
        # than our benchmark. For example, if the benchmark yield is 1 ETH and 2000 USDC, we will swap so that
        # we are at 1.03 ETH and 2060 USDC (approximately)
        # https://www.wolframalpha.com/input?i2d=true&i=solve+for+d+in+%5C%2840%29s+-+d%5C%2841%29b++%3D+%5C%2840%29t+%2B++Divide%5B%5C%2840%291+-+0.003%5C%2841%29+*+d+*+y%2Cx%5D+%5C%2841%29a
        will_swap_token1_for_token2 = withdraw_token1_amount > lending_token1_out and \
            withdraw_token1_amount / lending_token1_out > withdraw_token2_amount / lending_token2_out
        will_swap_token2_for_token1 = withdraw_token2_amount > lending_token2_out and \
            withdraw_token2_amount / lending_token2_out > withdraw_token1_amount / lending_token1_out
        if (will_swap_token1_for_token2):
            # This equation is a good approximation (just doesn't account for price impact,
            # so we very reasonably assume the swap is small compared to the size of the pool),
            # but we should sanity check it before doing the swap
            numer = 1000 * pool_token1_final_amount * \
                (lending_token2_out * withdraw_token1_amount - lending_token1_out * withdraw_token2_amount)
            denom = (997 * lending_token1_out * pool_token2_final_amount +
                    1000 * lending_token2_out * pool_token1_final_amount)
            swap_token1_amount_in = numer / denom
            # We will query pool balances in the smart contract call before doing the swap,
            # so there must be zero slippage
            estimated_token2_amount_out = 0.997 * swap_token1_amount_in * pool_token2_final_amount \
                / (pool_token1_final_amount + swap_token1_amount_in)
            withdraw_token1_amount -= swap_token1_amount_in
            withdraw_token2_amount += estimated_token2_amount_out
            logger.info(f'Rebalancing mechanism: We swap {np.round(swap_token1_amount_in, 2)} {token1_name} '
                f'for {np.round(estimated_token2_amount_out, 2)} {token2_name}')
        elif will_swap_token2_for_token1:
            numer = 1000 * pool_token2_final_amount * \
                (lending_token1_out * withdraw_token2_amount - lending_token2_out * withdraw_token1_amount)
            denom = (997 * lending_token2_out * pool_token1_final_amount +
                    1000 * lending_token1_out * pool_token2_final_amount)
            swap_token2_amount_in = numer / denom
            # We will query pool balances in the smart contract call before doing the swap,
            # so there must be zero slippage
            estimated_token1_amount_out = 0.997 * swap_token2_amount_in * pool_token1_final_amount \
                / (pool_token2_final_amount + swap_token2_amount_in)
            withdraw_token1_amount += estimated_token1_amount_out
            withdraw_token2_amount -= swap_token2_amount_in
            logger.info(f'Rebalancing mechanism: We swap {np.round(swap_token2_amount_in, 2)} {token2_name} '
                f'for {np.round(estimated_token1_amount_out, 2)} {token1_name}')
        else:
            logger.info('Unexpected, the benchmark performed better on both tokens. '
                'Is the DEX APY higher than the lending APY?')
            assert(false)
        logger.info(f'After the rebalancing mechanism, Alice withdraws '
        f'{np.round(withdraw_token1_amount, 2)} {token1_name}'
        f' and Bob withdraws {np.round(withdraw_token2_amount, 2)} {token2_name} from SyncSwap\n')
        logger.info('---------------')
        logger.info(f'Alice received {(withdraw_token1_amount / lending_token1_out - 1) * 100}% '
            f'more {token1_name} via single-sided SyncSwap LP than with ReactorFusion lending {token1_name} alone.')
        logger.info(f'Bob received {(withdraw_token2_amount / lending_token2_out - 1) * 100}% '
        f'more {token2_name} via single-sided SyncSwap LP than with ReactorFusion lending {token2_name} alone.')
        logger.info('---------------\n')
        misc_data = {
            "withdraw_token1_amount": withdraw_token1_amount,
            "unbalanced_withdraw_token1_amount": unbalanced_token1_amount,
            "withdraw_token2_amount": withdraw_token2_amount,
            "unbalanced_withdraw_token2_amount": unbalanced_token2_amount,
            "final_price": final_price
        }
        token1_dollar_value_over_dual_lp = (withdraw_token1_amount - unbalanced_token1_amount)
        token2_dollar_value_over_dual_lp = withdraw_token2_amount / unbalanced_token2_amount * final_price
        return withdraw_token1_amount / lending_token1_out - 1, withdraw_token2_amount / lending_token2_out - 1, \
            withdraw_token1_amount / user_token1_lp_amount - 1, withdraw_token2_amount / user_token2_lp_amount - 1, \
            misc_data
    
    simulate(usdc_pool_0, eth_pool_0, usdc_lp_amount, eth_lp_amount,
          usdc_pool_f_arr[0], eth_pool_f_arr[0],
          syncswap_apr, reactorfusion_usdc_apr, reactorfusion_eth_apr,
          "USDC", "ETH")
    

lping_vs_lending_analysis()
    

INFO     Initial SyncSwap ETH/USDC price = 1875.0


INFO     Pool TVL = $60.0 million

INFO     User 1 LPs 1 ETH. User 2 LPs 1875.0 USDC.
INFO     Users together own 0.006249609399412537% of the total pool
INFO     First few simulated end results:
INFO     USDC pool amounts = [18814855. 24135141. 29466320. 35218753.]
INFO     ETH pool amounts = [15949. 19183. 14597. 11509.]
INFO     ETH/USDC prices = [1180. 1258. 2019. 3060.]
INFO     Pool TVLs = $[37.62971  48.270282 58.93264  70.437506] million
INFO     Initial pool composition: 30000000 USDC, 16000 ETH
INFO     Initial price of ETH/USDC = 1875.0

INFO     1. Alice and Bob LP 1875.0 USDC and 1 ETH
INFO        Initially, they own 0.006249609399412537% of the total pool
INFO     2. Other users removed liquidity: 6281704.69 USDC, 3350.24 ETH
INFO     3. Other users bought 4905315.31 USDC for 3298.0 ETH

INFO     Final pool composition: 18814855.0 USDC, 15949.0 ETH
INFO     Final price of ETH/USDC = 1179.69
INFO     Ultimately, the specified LPs own 0.007904664998480842% of the total pool

## Price Impact Analysis

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def price_impact_analysis(
    weeks=52,
    initial_weekly_transactions=1000,
    initial_circulating_supply=1000000,
    initial_price=1.0,
    growth_rate=0.05,
    growth_rate_decay=0.01,
    buy_transactions=0.1,
    sell_transactions=0.05,
    D0=10000,
    kc=0.1,
    kl=0.05,
    Delta_Dc=500,
    Delta_Dl=300,
    epsilon_d=0.8,
    epsilon_s=0.5
):
    # Initialize data arrays
    weekly_transactions = np.zeros(weeks)
    tokens_locked = np.zeros(weeks)
    tokens_released = np.zeros(weeks)
    net_supply_change = np.zeros(weeks)
    circulating_supply = np.zeros(weeks)
    token_price = np.zeros(weeks)
    net_demand_change = np.zeros(weeks)
    price_change = np.zeros(weeks)
        
    # Initial conditions
    weekly_transactions[0] = initial_weekly_transactions
    circulating_supply[0] = initial_circulating_supply
    token_price[0] = initial_price
    net_demand_change[0] = D0 + kc * Delta_Dc + kl * Delta_Dl
    price_change[0]=0

    # Model each week
    for week in range(1, weeks):

        weekly_transactions[week] = weekly_transactions[week - 1] * (1 + growth_rate)
        tokens_locked[week] = weekly_transactions[week] * buy_transactions
        tokens_released[week] = weekly_transactions[week] * sell_transactions
        net_supply_change[week] = tokens_released[week] - tokens_locked[week] 
        circulating_supply[week] = circulating_supply[week - 1] + net_supply_change[week]

        # Recalculate demand increase for the week
        Delta_Dc *= (1 + growth_rate)  
        Delta_Dl *= (1 + growth_rate)  
        net_demand_change[week] = D0 + kc * Delta_Dc + kl * Delta_Dl

        # Price adjustment based on the change in demand and circulating supply
        
        delta_D = net_demand_change[week] - net_demand_change[week - 1]
        delta_S = net_supply_change[week]  
        
        # Price adjustment using price elasticities
        price_change_due_to_demand = (delta_D / (net_demand_change[week - 1] * epsilon_d))
        price_change_due_to_supply = (delta_S / ((circulating_supply[week - 1] + delta_S) * epsilon_s))

        # Net price change and new price calculation
        price_change[week] = price_change_due_to_demand + price_change_due_to_supply
        token_price[week] = token_price[week - 1] * (1 + price_change[week])

        # Growth rate decay on weekly basis
        growth_rate = growth_rate * (1 - week * growth_rate_decay)

    # Update results DataFrame with new calculations
    updated_results = pd.DataFrame({
        "Week": np.arange(weeks),
        "Transactions": weekly_transactions,
        "Tokens Locked": tokens_locked,
        "Tokens Released": tokens_released,
        "Weekly Supply Change": net_supply_change,
        "Weekly Demand Change": net_demand_change,
        "Circulating Supply": circulating_supply,
        "Percentage change Price": price_change,
        "Token Price": token_price,
    })

    # Plotting the results with the revised price adjustment
    fig, axs = plt.subplots(4, 1, figsize=(10, 15))

    # Plot for Circulating Supply
    axs[0].plot(updated_results['Week'], updated_results['Circulating Supply'], label='Circulating Supply', color='blue')
    axs[0].set_title('Effective Circulating Supply Over Time')
    axs[0].set_xlabel('Week')
    axs[0].set_ylabel('Effective Circulating Supply')
    axs[0].grid(True)

    # Plot for Token Price
    axs[1].plot(updated_results['Week'], updated_results['Token Price'], label='Token Price', color='red')
    axs[1].set_title('Token Price Over Time')
    axs[1].set_xlabel('Week')
    axs[1].set_ylabel('Token Price (USD)')
    axs[1].grid(True)

    # Plot for Demand Increase
    axs[2].plot(updated_results['Week'], updated_results['Weekly Demand Change'], label='Weekly Demand Change', color='green')
    axs[2].set_title('Weekly Demand Change Over Time')
    axs[2].set_xlabel('Week')
    axs[2].set_ylabel('Demand Change')
    axs[2].grid(True)

    # Plot for Tokens Locked
    axs[3].plot(updated_results['Week'], updated_results['Tokens Locked'], label='Tokens Locked', color='green')
    axs[3].set_title('Token Locked in Sinks Over Time')
    axs[3].set_xlabel('Week')
    axs[3].set_ylabel('Tokens Locked')
    axs[3].grid(True)

    plt.tight_layout()
    plt.show()

    return updated_results

# Example usage:



In [ ]:
results = price_impact_analysis()
print(results)

# Simulation Tools

## ILP Framework

In [54]:
import requests
import json

def initialize_script(base_path, reset_env_var=True):
    url = 'http://127.0.0.1:8000/initialize_script/'
    headers = {'Content-Type': 'application/json'}
    data = {
        "base_path": base_path,
        "reset_env_var": reset_env_var
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

def train_ddpg(max_steps=2, n_episodes=2, model_name="model_storage/ddpg/ddpg_fazool",
               alpha=0.001, beta=0.001, tau=0.8, batch_size=50, training=True,
               agent_budget_usd=10000, use_running_statistics=False):
    url = 'http://127.0.0.1:8000/train_ddpg/'
    headers = {'Content-Type': 'application/json'}
    data = {
        "max_steps": max_steps,
        "n_episodes": n_episodes,
        "model_name": model_name,
        "alpha": alpha,
        "beta": beta,
        "tau": tau,
        "batch_size": batch_size,
        "training": training,
        "agent_budget_usd": agent_budget_usd,
        "use_running_statistics": use_running_statistics
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

def evaluate_ddpg(eval_steps=2, eval_episodes=2, model_name="model_storage/ddpg/ddpg_fazool",
                  percentage_range=0.6, agent_budget_usd=10000, use_running_statistics=False):
    url = 'http://127.0.0.1:8000/evaluate_ddpg/'
    headers = {'Content-Type': 'application/json'}
    data = {
        "eval_steps": eval_steps,
        "eval_episodes": eval_episodes,
        "model_name": model_name,
        "percentage_range": percentage_range,
        "agent_budget_usd": agent_budget_usd,
        "use_running_statistics": use_running_statistics
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

def train_ppo(max_steps=2, n_episodes=2, model_name="model_storage/ppo/ppo2_fazool22",
              buffer_size=5, n_epochs=20, gamma=0.5, alpha=0.001, gae_lambda=0.75,
              policy_clip=0.6, max_grad_norm=0.6, agent_budget_usd=10000, use_running_statistics=False,
              action_transform="linear"):
    url = 'http://127.0.0.1:8000/train_ppo/'
    headers = {'Content-Type': 'application/json'}
    data = {
        "max_steps": max_steps,
        "n_episodes": n_episodes,
        "model_name": model_name,
        "buffer_size": buffer_size,
        "n_epochs": n_epochs,
        "gamma": gamma,
        "alpha": alpha,
        "gae_lambda": gae_lambda,
        "policy_clip": policy_clip,
        "max_grad_norm": max_grad_norm,
        "agent_budget_usd": agent_budget_usd,
        "use_running_statistics": use_running_statistics,
        "action_transform": action_transform
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

def evaluate_ppo(eval_steps=2, eval_episodes=2, model_name="model_storage/ppo/ppo2_fazool",
                 percentage_range=0.5, agent_budget_usd=10000, use_running_statistics=False,
                 action_transform="linear"):
    url = 'http://127.0.0.1:8000/evaluate_ppo/'
    headers = {'Content-Type': 'application/json'}
    data = {
        "eval_steps": eval_steps,
        "eval_episodes": eval_episodes,
        "model_name": model_name,
        "percentage_range": percentage_range,
        "agent_budget_usd": agent_budget_usd,
        "use_running_statistics": use_running_statistics,
        "action_transform": action_transform
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

def inference(pool_state, user_preferences, pool_id="0x4e68ccd3e89f51c3074ca5072bbac773960dfa36",
              ddpg_agent_path="model_storage/ddpg/ddpg_1",
              ppo_agent_path="model_storage/ppo/lstm_actor_critic_batch_norm"):
    url = 'http://127.0.0.1:8000/inference/'
    headers = {'Content-Type': 'application/json'}
    data = {
        "pool_state": pool_state,
        "user_preferences": user_preferences,
        "pool_id": pool_id,
        "ddpg_agent_path": ddpg_agent_path,
        "ppo_agent_path": ppo_agent_path
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()



{'error': "[Errno 2] No such file or directory: '/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/Intelligent-Liquidity-Provisioning-Framework-V1'"}
{'error': "'list' object has no attribute 'tolist'"}
{'error': "'list' object has no attribute 'tolist'"}
{'error': "'list' object has no attribute 'tolist'"}
{'error': "'list' object has no attribute 'tolist'"}
{'strategy_action': 'Add new liquidity position', 'ddpg_action': {'price_lower': 2936.108644564212, 'price_upper': 4536.0468162693805}, 'ppo_action': {'price_lower': 3044.989546812123, 'price_upper': 3317.2662559559903}}


In [ ]:
# Example usage:
base_path = "/mnt/d/Code/tempest/Intelligent-Liquidity-Provisioning-Framework-V2"
initialize_response = initialize_script(base_path)
print(initialize_response)

train_ddpg_response = train_ddpg()
print(train_ddpg_response)

evaluate_ddpg_response = evaluate_ddpg()
print(evaluate_ddpg_response)

train_ppo_response = train_ppo()
print(train_ppo_response)

evaluate_ppo_response = evaluate_ppo()
print(evaluate_ppo_response)

pool_state = {
    "current_profit": 500,
    "price_out_of_range": False,
    "time_since_last_adjustment": 40000,
    "pool_volatility": 0.2
}
user_preferences = {
    "risk_tolerance": {"profit_taking": 50, "stop_loss": -500},
    "investment_horizon": 7,
    "liquidity_preference": {"adjust_on_price_out_of_range": True},
    "risk_aversion_threshold": 0.1,
    "user_status": "new_user"
}
inference_response = inference(pool_state, user_preferences)
print(inference_response)


## Uniswap V3 simulator